In [ ]:
# Specify the path to your CSV file containing NIFTI paths
input_csv_path = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/05c_Howard_MetaAlzheimerReview_DBS-TMS_Coordinates/metadata/master_list.xlsx'
sheet = 'Sheet1' #'master_list_proper_subjects' ?

In [ ]:
from calvin_utils.permutation_analysis_utils.statsmodels_palm import CalvinStatsmodelsPalm
# Instantiate the PalmPrepararation class
cal_palm = CalvinStatsmodelsPalm(input_csv_path=input_csv_path, output_dir=None, sheet=sheet)
# Call the process_nifti_paths method
data_df = cal_palm.read_and_display_data()
data_df


# 01 - Preprocess Your Data

**Handle NANs**
- Set drop_nans=True is you would like to remove NaNs from data
- Provide a column name or a list of column names to remove NaNs from

In [ ]:
data_df.columns

In [ ]:
drop_list = ['cohens_D_redo']

In [ ]:
data_df = cal_palm.drop_nans_from_columns(columns_to_drop_from=drop_list)
data_df

**Drop Row Based on Value of Column**

Define the column, condition, and value for dropping rows
- column = 'your_column_name'
- condition = 'above'  # Options: 'equal', 'above', 'below'

Set the parameters for dropping rows

In [ ]:
data_df.columns

In [ ]:
column = 'Included'  # The column you'd like to evaluate
condition = 'not'  # Thecondition to check ('equal', 'above', 'below', 'not')
value = 1 # The value to compare against

In [ ]:
data_df, other_df = cal_palm.drop_rows_based_on_value(column, condition, value)
data_df

Regress out Covariates

In [ ]:
data_df.columns

In [ ]:
dependent_variable_list = ['cohens_D_redo']
regressors = ['Stimulation_site', 'Frequency']

In [ ]:
#Impute covariates in if desired
for col in regressors:
    if data_df[col].dtype == 'O':  # object/categorical
        data_df[col] = data_df[col].fillna(data_df[col].mode()[0])
    else:  # numeric
        data_df[col] = data_df[col].fillna(data_df[col].mean())
data_df[regressors].isna().sum()

In [ ]:
from calvin_utils.statistical_utils.regression_utils import RegressOutCovariates
data_df, adjusted_dep_vars_list = RegressOutCovariates.run(df=data_df, dependent_variable_list=dependent_variable_list, covariates_list=regressors)
print(adjusted_dep_vars_list)

**Standardize Data**
- Enter Columns you Don't want to standardize into a list

In [ ]:
data_df.columns

In [ ]:
# Remove anything you don't want to standardize
cols_not_to_standardize = ['TOTAL11']

In [ ]:
# data_df = cal_palm.standardize_columns(cols_not_to_standardize)

Do Miscellaneous Data Cleaning

In [ ]:
# data_df = data_df[(data_df['Pre_Post_Memory_Effect_Size__Cohen_s_D_'] >= -1) & (data_df['Pre_Post_Memory_Effect_Size__Cohen_s_D_'] <= 1)]

In [ ]:
data_df.columns

In [ ]:
import numpy as np
data_df['spatial_sign'] = np.where(data_df['CNM_Damage_Roi_old'] > 0, 'Positive', 'Negative')

# A - Simple BoxPLot

In [ ]:
data_df.columns

In [ ]:
category_col = 'spatial_sign'
y_col = 'cohens_D_redo_residual'

In [ ]:
out_dir = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/ccm_memory/results/tms_meta_analysis/redo_for_simon'

In [ ]:
from calvin_utils.statistical_utils.boxplot import BoxJitterPlotter
bjp = BoxJitterPlotter(figsize=(6,6))
bjp.plot(data_df, group_col=category_col, y_col=y_col, title='', out_dir=out_dir, xlabel="Cohen's D Sign", ylabel="Spatial Correlation")

In [ ]:
from scipy.stats import ttest_ind

# Get the unique groups
groups = data_df[category_col].unique()
if len(groups) != 2:
    raise ValueError("T-test requires exactly two groups.")

# Split the data by group
group1 = data_df[data_df[category_col] == groups[0]][y_col].dropna()
group2 = data_df[data_df[category_col] == groups[1]][y_col].dropna()

# Run independent t-test
ttest_ind(group1, group2)

In [ ]:
data_df.columns

In [ ]:
import numpy as np
np.std(obs.values-pred.values)

In [ ]:
len(obs)

In [ ]:
data_df.columns

In [ ]:
obs = data_df[data_df['optimizer'] == 0]['actual_score']
pred = data_df[data_df['optimizer'] == 1]['predictions']

t_stat, p_value = ttest_ind(obs, pred, equal_var=False)
print(f"T-statistic: {t_stat:.4f}, p-value: {p_value:.4e}")

In [ ]:
import numpy as np
from scipy.stats import t

# Means and standard deviations for the two groups
mean1 = np.mean(obs)
std1 = np.std(obs)
n1 = len(group1)

mean2 = 21
std2 = 21/np.sqrt(26)
n2 = len(group2)

# Calculate t-statistic for independent samples (unequal variance)
se = np.sqrt(std1**2 / n1 + std2**2 / n2)
t_stat_manual = (mean1 - mean2) / se

# Degrees of freedom (Welch-Satterthwaite equation)
df = (std1**2 / n1 + std2**2 / n2)**2 / (
    (std1**2 / n1)**2 / (n1 - 1) + (std2**2 / n2)**2 / (n2 - 1)
)

# Two-tailed p-value
p_value_manual = 2 * t.sf(np.abs(t_stat_manual), df)

print(f"Manual t-statistic: {t_stat_manual:.4f}, p-value: {p_value_manual:.4e}")

In [ ]:
(( (25*(105)**2) + (8 * 48**2) ) / 33)**0.5 / (35)**0.5

In [ ]:
16 * (9**0.5)

# B - Horizontal Box Plot
- Import a CSV with labels in the first column and observations across the other columns. Each row gets a boxplot. 

In [ ]:
data_path =''

In [ ]:
import pandas as pd
df = pd.read_csv(data_path)

In [ ]:
from calvin_utils.statistical_utils.boxplot import plot_horizontal_boxplot_with_stats
plot_horizontal_boxplot_with_stats(
    df,
    xlabel="x",
    out_dir=out_dir
)

Simple boxplot from CSV